In [1]:
import os
import pickle
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Union
import numpy as np
import pandas as pd
import scipy.optimize as sp
import math
import matplotlib.pyplot as plt

In [2]:
data_path = "../data/04_cricket_1999to2011.csv"

In [3]:
def get_data(data_path) -> Union[pd.DataFrame, np.ndarray]:
    """
    Loads the data from the given path and returns a pandas dataframe.

    Args:
        path (str): Path to the data file.

    Returns:
        pd.DataFrame, np.ndarray: Data Structure containing the loaded data
        
    """
    try:
        data = pd.read_csv(data_path)
        return data
    except FileNotFoundError:
        print(f"File not found at '{data_path}'. Please provide the correct path.")
        return None
    
    

In [4]:
data = get_data(data_path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126768 entries, 0 to 126767
Data columns (total 38 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Match                      126768 non-null  int64  
 1   Date                       126768 non-null  object 
 2   Innings                    126768 non-null  int64  
 3   Over                       126768 non-null  int64  
 4   Runs                       126768 non-null  int64  
 5   Total.Runs                 126768 non-null  int64  
 6   Innings.Total.Runs         126768 non-null  int64  
 7   Runs.Remaining             126768 non-null  int64  
 8   Total.Out                  126768 non-null  int64  
 9   Innings.Total.Out          126768 non-null  int64  
 10  Outs.Remaining             126768 non-null  int64  
 11  Wickets.in.Hand            126768 non-null  int64  
 12  Run.Rate                   126768 non-null  float64
 13  Innings.Run.Rate           12

In [5]:
data['Total.Overs'].value_counts()

50    126768
Name: Total.Overs, dtype: int64

In [6]:
data['Wickets.in.Hand'] == data['Outs.Remaining']

0         True
1         True
2         True
3         True
4         True
          ... 
126763    True
126764    True
126765    True
126766    True
126767    True
Length: 126768, dtype: bool

In [7]:
print(f"Are there any Null Values in dataset? ---- {data.isnull().values.any()}")
print(f"Total null values in dataset? ---- {data.isnull().sum().sum()}")
print(len(data))

Are there any Null Values in dataset? ---- True
Total null values in dataset? ---- 1109
126768


In [8]:
# data = data.dropna()

In [9]:
print(f"Are there any Null Values in dataset? ---- {data.isnull().values.any()}")
print(f"Total null values in dataset? ---- {data.isnull().sum().sum()}")
print(len(data))

Are there any Null Values in dataset? ---- True
Total null values in dataset? ---- 1109
126768


In [ ]:
column_names = list(data.columns.values)
column_names

In [ ]:
data['Error.In.Data'].isnull().values.any()
print(data['Date']).count_values().unique()

In [ ]:
def check_null_values(dataset):
    df = pd.DataFrame(dataset)
    null_info = {}

    for column in df.columns:
        null_count = df[column].isnull().sum()
        null_info[column] = null_count

    return null_info

import matplotlib.pyplot as plt

def dict_plot(null_info):
    mylist = [key for key, val in null_info.items() for _ in range(val)]
    plt.hist(mylist, bins=20)
    plt.show()


In [ ]:
null_info = check_null_values(data)
dict_plot(null_info)
print(f"Only Required-Run-Rate has NaN Values")

In [ ]:
#data = data.dropna()

In [ ]:
def null_details(data: Union[pd.DataFrame, np.ndarray]):
    print(f"Are there any Null Values in dataset? ---- {data.isnull().values.any()}")
    print(f"Total null values in dataset? ---- {data.isnull().sum().sum()}")
    print(f"The size of our dataset is {len(data)}")
    print(f"The size of our dataset is {len(data.columns)}")

In [ ]:
def null_details(data: Union[pd.DataFrame, np.ndarray]):
    print(f"Are there any Null Values in dataset? ---- {data.isnull().values.any()}")
    print(f"Total null values in dataset? ---- {data.isnull().sum().sum()}")
    print(f"The size of our dataset is {len(data)}")
    print(f"The size of our dataset is {len(data.columns)}")
    

def select_columns(data: Union[pd.DataFrame, np.ndarray], columns_to_keep):
    if isinstance(data, np.ndarray):
        data = pd.DataFrame(data)

    data = data[columns_to_keep]

    return data

def preprocess_data(data: Union[pd.DataFrame, np.ndarray]) -> Union[pd.DataFrame, np.ndarray]:
    """Preprocesses the dataframe by
    (i)   removing the unnecessary columns,
    (ii)  loading date in proper format DD-MM-YYYY,
    (iii) removing the rows with missing values,
    (iv)  anything else you feel is required for training your model.

    Args:
        data (pd.DataFrame, nd.ndarray): Pandas dataframe containing the loaded data

    Returns:
        pd.DataFrame, np.ndarray: Datastructure containing the cleaned data.
    """
    columns_to_keep = ['Innings', 'Innings.Total.Runs', 'Total.Overs', 'Wickets.in.Hand', 'Over', 'Total.Runs']
    
    print("Details before Preprocessing")
    print("-"*40)
    null_details(data)
    data = data.dropna()
    data = select_columns(data, columns_to_keep)
    print("Details After Preprocessing")
    print("-"*40)

    null_details(data)
    
    return data

In [ ]:
data = preprocess_data(data)

In [ ]:
len(data[data['Innings'] == 1])

In [ ]:
X = pd.DataFrame()
X["innings_number"]    = data['Innings'].values
X["remaining_runs"]  = data['Innings.Total.Runs'].values - data['Total.Runs'].values
X["remaining_overs"]   = data['Total.Overs'].values - data['Over'].values
X["wickets_in_hand"]   = data['Wickets.in.Hand'].values


In [ ]:
def plotparam_expectedrunvsoverremains(optparameters):
    '''
    This Procedure will plot the graph of ExpectedRun vs OverRemaining for all parameters.
    :param optparameters:
    This procedure will save the plot in source directory as "parameterplot_expectedrun_vs_overremain.png"
    '''
    plt.figure(1)
    plt.title("Expected Runs vs Overs Remaininng")
    plt.xlim((0, 50))
    plt.ylim((0, 250))
    plt.xticks([0, 10, 20, 30, 40, 50])
    plt.yticks([0, 50, 100, 150, 200, 250])
    plt.xlabel('Overs remaining')
    plt.ylabel('Expected Runs')
    colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', '#555b65', '#999e45', '#222a55']
    x=np.zeros((51))
    for i in range(51):
        x[i]=i
    for i in range(len(optparameters)-1):
        y_run=optparameters[i] * (1 - np.exp(-optparameters[10] * x /optparameters[i]))
        plt.plot(x, y_run, c=colors[i], label='Z[' + str(i + 1) + ']')
        plt.legend()
    #plt.savefig('parameterplot_expectedrun_vs_overremain.png')
    plt.show()
    #plt.close()

In [ ]:
def plotparam_resourceremainvsoverremains(optparameters):
    '''
        This Procedure will plot the graph of ResourceRemainings vs OverRemaining for all parameters.
        :param optparameters:
        This procedure will save the plot in source directory as "parameterplot_resourceremain_vs_overremain.png"
        '''
    plt.figure(1)
    plt.title("Resource Remaining vs Overs Remaininng")
    plt.xlim((0, 50))
    plt.ylim((0, 100))
    plt.xticks([0, 10, 20, 30, 40, 50])
    plt.yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    plt.xlabel('Overs remaining')
    plt.ylabel('percentage Of Resource Remaining')
    colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', '#555b65', '#999e45', '#222a55']
    x = np.zeros((51))
    for i in range(51):
        x[i] = i
    Z5010=optparameters[9] * (1 - np.exp(-optparameters[10] * 50 /optparameters[9]))
    for i in range(len(optparameters)-1):
        y_run=optparameters[i] * (1 - np.exp(-optparameters[10] * x /optparameters[i]))
        plt.plot(x, (y_run/Z5010)*100, c=colors[i], label='Z[' + str(i + 1) + ']')
        plt.legend()
    #plt.savefig('parameterplot_resourceremain_vs_overremain.png')
    plt.show()
    #plt.close()
    

In [ ]:
def fit_parameters(data):
    '''
    This procedure will fit the curve to optimise the overall loss function against 11 parameters.
    :param innings_number:
    :param runs_scored:
    :param remaining_overs:
    :param wickets_in_hand:
    :return:optimised_res['fun']:Total Loss incurred
    :return:optimised_res['x']:Optimised values of all 11 parameters.
    '''
    parameters = [10, 30, 40, 60, 90, 125, 150, 170, 190, 200,10]
    
    innings_number    = data['Innings'].values
    remaining_runs    = data['Innings.Total.Runs'].values - data['Total.Runs'].values
    remaining_overs   = data['Total.Overs'].values - data['Over'].values
    wickets_in_hand   = data['Wickets.in.Hand'].values
    
    print(f"are lenght of over remaining values equal to wicket in hand : {len(remaining_overs) == len(wickets_in_hand)}")
    optimised_res = sp.minimize(sum_of_squared_errors_loss_function,parameters,
                      args=[innings_number, remaining_runs, remaining_overs, wickets_in_hand],
                      method='L-BFGS-B')
    return optimised_res['fun'],optimised_res['x']

def sum_of_squared_errors_loss_function(parameters,args):
    '''
    This procedure defines the objective function which I have passed in scipy.optimize.minimize() function.
    It calculated all total squared error loss for all the data points for innings 1.
    :param parameters: List contains 11 parameters
    :param args: List contains innings_number,runs_scored,remaining_overs,wickets_in_hand
    :return:total_squared_error of the objective function.
    '''
    total_squared_error=0
    l_param=parameters[10]
    innings_number = args[0]
    runs_scored=args[1]
    remaining_overs=args[2]
    wickets_in_hand=args[3]
    for i in range(len(wickets_in_hand)):
        if innings_number[i] == 1:
            runscored = runs_scored[i]
            overremain = remaining_overs[i]
            wicketinhand = wickets_in_hand[i]
            Z0=parameters[wicketinhand - 1]
            if runscored > 0:
                predicted_run =  Z0 * (1 - np.exp(-1*l_param * overremain / Z0))
                total_squared_error=total_squared_error + (math.pow(predicted_run - runscored, 2))
    return total_squared_error

In [ ]:
loss, parameeeee = fit_parameters(data)

In [ ]:
plotparam_resourceremainvsoverremains(parameeeee)

In [ ]:
plotparam_expectedrunvsoverremains(parameeeee)

In [ ]:
for i in range(len(parameeeee)):
        if(i == 10):
            print("L :"+str(parameeeee[i]))
        else:
            print("Z["+str(i+1)+"] :"+str(parameeeee[i]))

print(f"Loss : {loss}")

In [ ]:
class DLModel:
    """
        Model Class to approximate the Z function as defined in the assignment.
    """

    def __init__(self):
        """Initialize the model."""
        self.Z0 = [None] * 10
        self.L = None
    
    def get_predictions(self, X, Z_0=None, w=10, L=None) -> np.ndarray:
        """Get the predictions for the given X values.

        Args:
            X (np.array): Array of overs remaining values.
            Z_0 (float, optional): Z_0 as defined in the assignment.
                                   Defaults to None.
            w (int, optional): Wickets in hand.
                               Defaults to 10.
            L (float, optional): L as defined in the assignment.
                                 Defaults to None.

        Returns:
            np.array: Predicted score possible
        """
        if Z_0 is None:
            Z_0 = self.Z0[w - 1]  # Using Z0 for given wickets
            
        if L is None:
            L = self.L
        
        predicted_scores = Z_0 * (1 - np.exp(-1 * L * X / Z_0))
        return predicted_scores

    def calculate_loss(self, Params, X, Y, w=10) -> float:
        """ Calculate the loss for the given parameters and datapoints.
        Args:
            Params (list): List of parameters to be optimized.
            X (np.array): Array of overs remaining values.
            Y (np.array): Array of actual average score values.
            w (int, optional): Wickets in hand.
                               Defaults to 10.

        Returns:
            float: Mean Squared Error Loss for the model parameters 
                   over the given datapoints.
        """
        total_squared_error = 0
        l_param = Params[-1]  # L value is the last parameter
        Z_values = Params[:-1]  # Extract Z0 values from parameters

        
#         for i in range(len(X)):
#             if X[i] == 1:
#                 predicted_run = Z_values[w - 1] * (1 - np.exp(-1 * l_param * X[i] / Z_values[w - 1]))
# #                 predicted_run = self.get_predictions(X, Z_values, w, l_param)
#                 total_squared_error += (Y[i] - predicted_run) ** 2
                
#         return total_squared_error
#         l_param=parameters[10]

        innings_number  = X["innings_number"]
        runs_scored     = X["remaining_runs"]
        remaining_overs = X["remaining_overs"]
        wickets_in_hand = X["wickets_in_hand"]
                            
        for i in range(len(wickets_in_hand)):
            if innings_number[i] == 1:
                runscored = runs_scored[i]
                overremain = remaining_overs[i]
                wicketinhand = wickets_in_hand[i]
                Z0=Y[wicketinhand - 1]
                if runscored > 0:
                    predicted_run =  Z0 * (1 - np.exp(-1*l_param * overremain / Z0))
                    total_squared_error=total_squared_error + (math.pow(predicted_run - runscored, 2))
        return total_squared_error
        
    
    def save(self, path):
        """Save the model to the given path.

        Args:
            path (str): Location to save the model.
        """
        with open(path, 'wb') as f:
            pickle.dump((self.L, self.Z0), f)
    
    def load(self, path):
        """Load the model from the given path.

        Args:
            path (str): Location to load the model.
        """
        with open(path, 'rb') as f:
            (self.L, self.Z0) = pickle.load(f)

def train_model(data: Union[pd.DataFrame, np.ndarray], model: DLModel):   #-> DLModel, list
    """Trains the model

    Args:
        data (pd.DataFrame, np.ndarray): Datastructure containg the cleaned data
        model (DLModel): Model to be trained
    """
    parameters = [10, 30, 40, 60, 90, 125, 150, 170, 190, 200,10]
    Y = [10, 30, 40, 60, 90, 125, 150, 170, 190, 200,10]
    
    X["innings_number"]    = data['Innings'].values
    X["remaining_runs"]    = data['Innings.Total.Runs'].values - data['Total.Runs'].values
    X["remaining_overs"]   = data['Total.Overs'].values - data['Over'].values
    X["wickets_in_hand"]   = data['Wickets.in.Hand'].values
    
#     argss=[innings_number, remaining_runs, remaining_overs, wickets_in_hand]
    
    
    
#     optimised_res = sp.minimize(model.calculate_loss, (parameters, argss[2], argss[1]),
#                       args=[innings_number, remaining_runs, remaining_overs, wickets_in_hand]
#                      ) # method='L-BFGS-B'
    
#     optimized_params = optimised_res['x']
#     model.Z0 = optimized_params[:-1]  # Assign optimized Z0 values
#     model.L = optimized_params[-1]     # Assign optimized L value
#     def loss_function(params, data):
#             return model.calculate_loss(params, data, parameters)
    
    # Optimize the parameters using some optimization algorithm (e.g., minimize)
    optimization_result = sp.minimize(model.calculate_loss, parameters, 
                                      args = (parameters, X, Y), method='L-BFGS-B')
    
    # L-BFGS-B, Powell, Nelder-Mead
    optimized_params = optimization_result['x']
    optimized_loss = optimization_result['fun']
        
    # Update the model with the optimized parameters
    model.L = optimized_params[-1]  # Update L value
    model.Z0 = optimized_params[:-1]  # Update Z0 values
    print("Total Loss:", optimized_loss)
#     print("Optimized Parameters:", optimized_params)
    for i in range(len(optimized_params)):
        if(i == 10):
            print("L :"+str(optimized_params[i]))
        else:
            print("Z["+str(i+1)+"] :"+str(optimized_params[i]))
    
    return model

In [ ]:
# model = DLModel()
# model = train_model(data, model)

In [ ]:
# model.save('../models/model.pkl')

In [ ]:
# plotparam_resourceremainvsoverremains(optimized_params)


In [ ]:
# plotparam_expectedrunvsoverremains(optimized_params)

In [ ]:
class DLModel:
    """
        Model Class to approximate the Z function as defined in the assignment.
    """

    def __init__(self):
        """Initialize the model."""
        self.Z0 = [None] * 10
        self.L = None
        self.loss = 0
    
    def get_predictions(self, X, Z_0=None, w=10, L=None) -> np.ndarray:
        """Get the predictions for the given X values.

        Args:
            X (np.array): Array of overs remaining values.
            Z_0 (float, optional): Z_0 as defined in the assignment.
                                   Defaults to None.
            w (int, optional): Wickets in hand.
                               Defaults to 10.
            L (float, optional): L as defined in the assignment.
                                 Defaults to None.

        Returns:
            np.array: Predicted score possible
        """
        if Z_0 is None:
            Z_0 = self.Z0[w - 1]  # Using Z0 for given wickets
            
        if L is None:
            L = self.L
        
        predicted_scores = Z_0 * (1 - np.exp(-1 * L * X / Z_0))
        return predicted_scores
        

    def calculate_loss(self, Params, X, Y, w=10) -> float:
        """ Calculate the loss for the given parameters and datapoints.
        Args:
            Params (list): List of parameters to be optimized.
            X (np.array): Array of overs remaining values.
            Y (np.array): Array of actual average score values.
            w (int, optional): Wickets in hand.
                               Defaults to 10.

        Returns:
            float: Mean Squared Error Loss for the model parameters 
                   over the given datapoints.
        """
        pass
    
    def save(self, path):
        """Save the model to the given path.

        Args:
            path (str): Location to save the model.
        """
        with open(path, 'wb') as f:
            pickle.dump((self.L, self.Z0), f)
    
    def load(self, path):
        """Load the model from the given path.

        Args:
            path (str): Location to load the model.
        """
        with open(path, 'rb') as f:
            (self.L, self.Z0) = pickle.load(f)

def train_model(data: Union[pd.DataFrame, np.ndarray], model: DLModel)-> DLModel:
    """Trains the model

    Args:
        data (pd.DataFrame, np.ndarray): Datastructure containg the cleaned data
        model (DLModel): Model to be trained
    """
    def fit_parameters(data):
        '''
        This procedure will fit the curve to optimise the overall loss function against 11 parameters.
        :param innings_number:
        :param runs_scored:
        :param remaining_overs:
        :param wickets_in_hand:
        :return:optimised_res['fun']:Total Loss incurred
        :return:optimised_res['x']:Optimised values of all 11 parameters.
        '''
        parameters = [10, 30, 40, 60, 90, 125, 150, 170, 190, 200,10]

        innings_number    = data['Innings'].values
        remaining_runs    = data['Innings.Total.Runs'].values - data['Total.Runs'].values
        remaining_overs   = data['Total.Overs'].values - data['Over'].values
        wickets_in_hand   = data['Wickets.in.Hand'].values

        # print(f"are lenght of over remaining values equal to wicket in hand : {len(remaining_overs) == len(wickets_in_hand)}")
        optimised_res = sp.minimize(sum_of_squared_errors_loss_function,parameters,
                          args=[innings_number, remaining_runs, remaining_overs, wickets_in_hand],
                          method='L-BFGS-B')
        return optimised_res['fun'],optimised_res['x']

    def sum_of_squared_errors_loss_function(parameters,args):
        '''
        This procedure defines the objective function which I have passed in scipy.optimize.minimize() function.
        It calculated all total squared error loss for all the data points for innings 1.
        :param parameters: List contains 11 parameters
        :param args: List contains innings_number,runs_scored,remaining_overs,wickets_in_hand
        :return:total_squared_error of the objective function.
        '''
        total_squared_error=0
        l_param=parameters[10]
        innings_number = args[0]
        runs_scored=args[1]
        remaining_overs=args[2]
        wickets_in_hand=args[3]
        for i in range(len(wickets_in_hand)):
            if innings_number[i] == 1:
                runscored = runs_scored[i]
                overremain = remaining_overs[i]
                wicketinhand = wickets_in_hand[i]
                Z0=parameters[wicketinhand - 1]
                if runscored > 0:
                    predicted_run =  Z0 * (1 - np.exp(-1*l_param * overremain / Z0))
                    total_squared_error=total_squared_error + (math.pow(predicted_run - runscored, 2))
        return total_squared_error
    
    
    loss_value, param = fit_parameters(data)
    
    model.L = param[-1]  # Update L value
    model.Z0 = param[:-1]  # Update Z0 valuues
    model.loss = loss_value



    print("Total Loss:", loss_value)

    # for i in range(len(param)):
    #     if(i == 10):
    #         print("L :"+str(param[i]))
    #     else:
    #         print("Z["+str(i+1)+"] :"+str(param[i]))
    
    return model


model = DLModel()
model = train_model(data, model)

In [ ]:
d

In [ ]:
print_model_params(model)

In [ ]:


print(parameeeee == optparameters)
print(optparameters)
print(parameeeee)

In [ ]:
print(len(parameeeee))
plt.figure(1)
plt.title("Expected Runs vs Overs Remaininng")
plt.xlim((0, 50))
plt.ylim((0, 250))
plt.xticks([0, 10, 20, 30, 40, 50])
plt.yticks([0, 50, 100, 150, 200, 250])
plt.xlabel('Overs remaining')
plt.ylabel('Expected Runs')
colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', '#555b65', '#999e45', '#222a55']
x=np.zeros((51))
for i in range(51):
    x[i]=i
for i in range(len(parameeeee)-1):
    y_run=parameeeee[i] * (1 - np.exp(-parameeeee[10] * x /parameeeee[i]))
    plt.plot(x, y_run, c=colors[i], label='Z[' + str(i + 1) + ']')
    plt.legend()
#plt.savefig('parameterplot_expectedrun_vs_overremain.png')
plt.show()
#plt.close()

In [ ]:
def plot(model: DLModel, plot_path: str) -> None:
    """ Plots the model predictions against the number of overs
        remaining according to wickets in hand.

    Args:
        model (DLModel): Trained model
        plot_path (str): Path to save the plot
    """
    Z0 = model.Z0
    L = model.L
    optparameters = np.insert(Z0, 10, L)
    print(len(optparameters))
    plt.figure(1)
    plt.title("Expected Runs vs Overs Remaininng")
    plt.xlim((0, 50))
    plt.ylim((0, 250))
    plt.xticks([0, 10, 20, 30, 40, 50])
    plt.yticks([0, 50, 100, 150, 200, 250])
    plt.xlabel('Overs remaining')
    plt.ylabel('Expected Runs')
    colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', '#555b65', '#999e45', '#222a55']
    x=np.zeros((51))
    for i in range(51):
        x[i]=i
    for i in range(len(optparameters)-1):
        y_run=optparameters[i] * (1 - np.exp(-optparameters[10] * x /optparameters[i]))
        plt.plot(x, y_run, c=colors[i], label='Z[' + str(i + 1) + ']')
        plt.legend()
    plt.savefig('plot_path')
    plt.show()
    plt.close()


In [ ]:
def plot_resource_remaining(model: DLModel, plot_path: str) -> None:
    """ Plots the model predictions against the number of overs
        remaining according to wickets in hand.

    Args:
        model (DLModel): Trained model
        plot_path (str): Path to save the plot
    """
    
    Z0 = model.Z0
    L = model.L
    optparameters = np.insert(Z0, 10, L)
    plt.figure(1)
    plt.title("Resource Remaining vs Overs Remaininng")
    plt.xlim((0, 50))
    plt.ylim((0, 100))
    plt.xticks([0, 10, 20, 30, 40, 50])
    plt.yticks([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    plt.xlabel('Overs remaining')
    plt.ylabel('percentage Of Resource Remaining')
    colors = ['r', 'g', 'b', 'y', 'c', 'm', 'k', '#555b65', '#999e45', '#222a55']
    x = np.zeros((51))
    for i in range(51):
        x[i] = i
    Z5010=optparameters[9] * (1 - np.exp(-optparameters[10] * 50 /optparameters[9]))
    for i in range(len(optparameters)-1):
        y_run=optparameters[i] * (1 - np.exp(-optparameters[10] * x /optparameters[i]))
        plt.plot(x, (y_run/Z5010)*100, c=colors[i], label='Z[' + str(i + 1) + ']')
        plt.legend()
    plt.savefig('parameterplot_resourceremain_vs_overremain.png')
    plt.show()
    plt.close()